In [1]:
import os
# Reemplaza con tu ruta si es diferente
os.add_dll_directory('C:\\Program Files\\IBM\\SQLLIB\\BIN')
# conectar a la base de datos IBM Db2
import ibm_db

In [2]:
import queries
import pandas as pd
# import ibm_db_dbi
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
conn = ibm_db.connect('DATABASE=' + os.getenv('DATABASE') + ';HOSTNAME=' + os.getenv('HOSTNAME') + ';PORT=' + os.getenv('PORT') + ';PROTOCOL=TCPIP;UID=' + os.getenv('USERNAME_DB') + ';PWD=' + os.getenv('PASSWORD_DB') + ';', '', '')
# conn = ibm_db.connect(os.getenv('DATABASE'), os.getenv('USERNAME_DB'), os.getenv('PASSWORD_DB')) # os.getenv('HOSTNAME'), os.getenv('PORT')

if conn:
    print("Conexión exitosa")
else:
    print("Error al conectar")

Conexión exitosa


In [4]:
cod_materia = "MATG1076"

In [5]:
# stmt_select  = ibm_db.exec_immediate(conn, queries.nota_materia)
stmt_select  = ibm_db.exec_immediate(conn, queries.get_reprobacion_materias_con_socioeconomico(cod_materia))

In [6]:
print("Number of affected rows: ", ibm_db.num_rows(stmt_select))

Number of affected rows:  -1


In [7]:
# # Fetch and print the column names
# column_names = []
# while ibm_db.fetch_row(stmt_select) != False:
#     column_name = ibm_db.result(stmt_select, 0) # Get the value from the first column (NAME)
#     column_names.append(column_name)

In [8]:
# tuple = ibm_db.fetch_tuple(stmt_select)
# while tuple != False:
#     print("The COD is : ", tuple[0])
#     print("The año : ", tuple[1])
#     tuple = ibm_db.fetch_tuple(stmt_select)

In [9]:
# Fetch all rows
data_list = []
result = ibm_db.fetch_assoc(stmt_select)
while result:
    # print(result)
    data_list.append(result)
    result = ibm_db.fetch_assoc(stmt_select)

In [10]:
# Close the connection
ibm_db.close(conn)

True

In [11]:
# data_list
# list to dataframe
df = pd.DataFrame(data_list)

In [12]:
df.to_csv("data/"+ cod_materia + "_reprobacion_materias_con_socioeconomico.csv", index=False)
print("Guardado con exito en", "'data/"+ cod_materia + "_reprobacion_materias_con_socioeconomico.csv'")

Guardado con exito en 'data/MATG1076_reprobacion_materias_con_socioeconomico.csv'
